In [23]:
from mmengine.fileio import list_from_file
import torch
import torchvision.transforms as transforms
import xmltodict
import mmcv
import os
import numpy as np

class_dict = {
    'aeroplane': 0,
    'bicycle': 1,
    'bird': 2,
    'boat': 3,
    'bottle': 4,
    'bus': 5,
    'car': 6,
    'cat': 7,
    'chair': 8,
    'cow': 9,
    'diningtable': 10,
    'dog': 11,
    'horse': 12,
    'motorbike': 13,
    'person': 14,
    'pottedplant': 15,
    'sheep': 16,
    'sofa': 17,
    'train': 18,
    'tvmonitor': 19
}

# 指定要遍历的文件夹路径
img_ids = list_from_file("../data/VOCdevkit/VOC2007/ImageSets/Main/test.txt")

annotations, det_results  = [], []
for j in range(len(img_ids)):

    # 解析XML文件
    ann_path = os.path.join('../data/VOCdevkit/VOC2007/Annotations/', img_ids[j] + '.xml')
    with open(ann_path) as f:
        xml_data = xmltodict.parse(f.read())
    bboxes, labels = [], []
    bboxes_ignore, labels_ignore = [], []
    obj = xml_data['annotation']['object']
    if type(obj) == list:
        for i in range(len(obj)):
            if obj[i]['difficult'] == '0':
                bboxes.append([int(obj[i]['bndbox']['xmin'])-1, int(obj[i]['bndbox']['ymin'])-1, 
                            int(obj[i]['bndbox']['xmax'])-1, int(obj[i]['bndbox']['ymax'])-1])
                labels.append(class_dict[obj[i]['name']])
            else:
                bboxes_ignore.append([int(obj[i]['bndbox']['xmin'])-1, int(obj[i]['bndbox']['ymin'])-1, 
                            int(obj[i]['bndbox']['xmax'])-1, int(obj[i]['bndbox']['ymax'])-1])
                labels_ignore.append(class_dict[obj[i]['name']])
    else:
        if obj['difficult'] == '0':
            bboxes.append([int(obj['bndbox']['xmin'])-1, int(obj['bndbox']['ymin'])-1, 
                        int(obj['bndbox']['xmax'])-1, int(obj['bndbox']['ymax'])-1])
            labels.append(class_dict[obj['name']])
        else:
            bboxes_ignore.append([int(obj['bndbox']['xmin'])-1, int(obj['bndbox']['ymin'])-1, 
                        int(obj['bndbox']['xmax'])-1, int(obj['bndbox']['ymax'])-1])
            labels_ignore.append(class_dict[obj['name']])

    bboxes = torch.tensor(bboxes).cpu().numpy().astype(np.float32)
    labels = torch.tensor(labels).cpu().numpy()
    bboxes_ignore = torch.tensor(bboxes_ignore).cpu().numpy().astype(np.float32)
    labels_ignore = torch.tensor(labels_ignore).cpu().numpy()

    ann = {'bboxes': bboxes,
           'labels': labels,
           'bboxes_ignore': torch.empty(size=(0,4)).cpu().numpy() if len(bboxes_ignore) == 0 else bboxes_ignore,
           'labels_ignore': torch.empty(dtype=torch.int64, size=(0,)).cpu().numpy() if len(labels_ignore) == 0 else labels_ignore}
    annotations.append(ann)

    
    # 加载图像并进行预处理
    preds_path = os.path.join('../data/VOCdevkit/VOC2007/test_img_model1/', img_ids[j] + '.txt')
    with open(preds_path, 'r') as f:
        preds = f.read()
    preds = eval(preds)
    
    scale_factor = (224.0/int(xml_data['annotation']['size']['width']),
                    224.0/int(xml_data['annotation']['size']['height']))
    
    pred_bboxes = np.empty(shape=(0,4))
    pred_scores = np.empty(shape=(0))
    pred_labels = np.empty(shape=(0))
    if preds is not None:
        for pred in preds:
            pred_bboxes = np.append(pred_bboxes, [[pred['x'], pred['y'], pred['x']+pred['w'], pred['y']+pred['h']]], axis=0)
            pred_scores = np.append(pred_scores, pred['value'])
            pred_labels = np.append(pred_labels, pred['classid'])

    pred_bboxes[:,[0,2]] /= scale_factor[0]
    pred_bboxes[:,[1,3]] /= scale_factor[1]    

    dets = []
    for label in range(len(class_dict)):
        index = np.where(pred_labels == label)[0]
        pred_bbox_scores = np.hstack(
            [pred_bboxes[index], pred_scores[index].reshape((-1, 1))])
        dets.append(pred_bbox_scores)
    
    det_results.append(dets)
    

In [24]:
from mmdet.evaluation.functional.mean_ap import eval_map

mean_ap, eval_results = eval_map(det_results, annotations, eval_mode='11points', use_legacy_coordinate=True)



+-------+------+-------+--------+-------+
| class | gts  | dets  | recall | ap    |
+-------+------+-------+--------+-------+
| 0     | 285  | 888   | 0.204  | 0.129 |
| 1     | 337  | 42    | 0.009  | 0.018 |
| 2     | 459  | 4450  | 0.113  | 0.017 |
| 3     | 263  | 1259  | 0.118  | 0.049 |
| 4     | 469  | 1598  | 0.013  | 0.001 |
| 5     | 213  | 31    | 0.023  | 0.091 |
| 6     | 1201 | 1433  | 0.117  | 0.096 |
| 7     | 358  | 5252  | 0.408  | 0.149 |
| 8     | 756  | 23529 | 0.194  | 0.008 |
| 9     | 244  | 478   | 0.020  | 0.010 |
| 10    | 206  | 1712  | 0.189  | 0.010 |
| 11    | 489  | 3565  | 0.194  | 0.058 |
| 12    | 348  | 498   | 0.147  | 0.083 |
| 13    | 325  | 33    | 0.022  | 0.091 |
| 14    | 4528 | 10394 | 0.164  | 0.110 |
| 15    | 480  | 1657  | 0.021  | 0.004 |
| 16    | 242  | 1015  | 0.050  | 0.023 |
| 17    | 239  | 4897  | 0.418  | 0.021 |
| 18    | 282  | 174   | 0.035  | 0.045 |
| 19    | 308  | 736   | 0.166  | 0.107 |
+-------+------+-------+--------+